In [ ]:
! pip uninstall -y transformer_lens
! pip install git+https://github.com/taufeeque9/TransformerLens/
! pip install git+https://github.com/minyoungg/vqtorch/
! pip install termcolor
! pip install -U accelerate
! pip install -U kaleido

In [28]:
from termcolor import colored
import plotly.express as px
import transformers
import codebook_features
import torch
import evaluate
import numpy as np
import copy
import wandb
import json
import transformer_lens.utils as utils
from collections import namedtuple
from functools import partial
from torch.nn import functional as F
import itertools
from tqdm import tqdm
import math
from datetime import datetime
import pickle
import glob


from transformers import (
    GPTNeoXConfig,
    GPTNeoXForCausalLM,
    GPT2TokenizerFast,
    pipeline,
    set_seed,
)
from torch.utils.data import IterableDataset
from codebook_features import models, run_clm, train_toy_model, trainer as cb_trainer   
from codebook_features.utils import *
from codebook_features.toy_utils import *
import os

torch.set_grad_enabled(False)

In [2]:
model_name_or_path = "EleutherAI/pythia-410m-deduped"
pretrained_path = "/data/outputs/2023-04-24/14-05-06/output_main/"
device = "cuda"
orig_cb_model = models.wrap_codebook(model_or_path=model_name_or_path, pretrained_path=pretrained_path)
orig_cb_model = orig_cb_model.to(device).eval()
orig_cb_model.disable_logging()
print("Loaded original cb model.")
hooked_kwargs = dict(center_unembed=False,fold_value_biases=False,center_writing_weights=True,fold_ln=True,refactor_factored_attn_matrices=False,device=device)
cb_model = models.convert_to_hooked_model(model_name_or_path, orig_cb_model, hooked_kwargs=hooked_kwargs)
# # cb_model = cb_model.model
cb_model.disable_logging()
cb_model = cb_model.to(device).eval()
model = orig_cb_model
tokenizer = cb_model.tokenizer

Time: 36.92286515235901
Time: 5.9604644775390625e-06 5.7697296142578125e-05 37.652931213378906
Loaded original cb model.


Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-410m-deduped into HookedTransformer
Time: 8.106231689453125e-06 5.8650970458984375e-05 25.994243144989014
Time: 77.78085708618164 0.08902430534362793 0.11349678039550781 25.995729207992554 0.3244805335998535


In [3]:
report_to = "none"
# report_to = "all"
training_args = run_clm.TrainingArguments(
    #     no_cuda=True,
    output_dir="/shared/output_attn_pythia410/",
    do_eval=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
)

model_args = run_clm.ModelArguments(model_name_or_path=pretrained_path, cache_dir="/data/.cache/huggingface/")
data_args = run_clm.DataTrainingArguments(
    dataset_name="wikitext", dataset_config_name="wikitext-103-v1", streaming=False,
)

trainer, lm_datasets, last_checkpoint = run_clm.get_trainer_and_dataset(
    model_args,
    data_args,
    training_args,
    model,
    optimizers=(None, None),
)

07/04/2023 19:06:26 - WARNING - codebook_features.run_clm - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False
07/04/2023 19:06:27 - WARNING - datasets.builder - Found cached dataset wikitext (/data/.cache/huggingface/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

07/04/2023 19:09:05 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /data/.cache/huggingface/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-f8d0d6b5b5fe7dc9.arrow
07/04/2023 19:12:14 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /data/.cache/huggingface/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-a53c183da4ec97cb.arrow
07/04/2023 19:12:15 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /data/.cache/huggingface/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-8183c2e143fa69fe.arrow
07/04/2023 19:12:15 - WARNING - codebook_features.run_clm - The tokenizer picked seems to have a very large `model_max_length` (1000000000000000019884624838656). Picking 1024 instead. You can change that default value by passing --block_size xxx.
07/04/2023 19:12:16 - WA

In [4]:
seed = 42
rng = np.random.default_rng(seed)
max_samples = 2000
dataset = lm_datasets["train"].select(rng.choice(len(lm_datasets["train"]), max_samples, replace=False))
tokens = dataset["input_ids"]

In [7]:
from datetime import datetime
import pickle
import glob

trainer.args.report_on = "none"
cb_acts = {}
ft_tkns = {}

def store_cb_activations(key, codebook_ids, codebook_acts=cb_acts):
    assert len(codebook_ids.shape) == 3  # (bs, seq_len, k_codebook)
    if key not in codebook_acts:
        codebook_acts[key] = []
    codebook_acts[key].append(codebook_ids)


output_dir = "/shared/cb_eval_acts/pythia410_preproj_attn*"
dirs = glob.glob(output_dir)
if len(dirs) > 0:
    print("Loading activations from", dirs[0])
    with open(f"{dirs[0]}/cb_acts.pkl", "rb") as f:
        cb_acts = pickle.load(f)
    tokens = np.load(f"{dirs[0]}/tokens.npy")
    metrics = np.load(f"{dirs[0]}/metrics.npy", allow_pickle=True)
else:
    orig_cb_model.set_hook_fn(store_cb_activations)
    metrics = trainer.evaluate(dataset)
    for k, v in cb_acts.items():
        cb_acts[k] = np.concatenate(v, axis=0)
    output_dir = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    output_dir = "/shared/cb_eval_acts/pythia410_preproj_attn_" + output_dir
    os.makedirs(output_dir, exist_ok=True)
    np.save(f"{output_dir}/tokens.npy", tokens)
    np.save(f"{output_dir}/metrics.npy", metrics)
    with open(f"{output_dir}/cb_acts.pkl", "wb") as f:
        pickle.dump(codebook_acts, f)

print(metrics)
num_codes = 10_000
cb_at = "attn_preproj"

Loading activations from /shared/cb_eval_acts/pythia410_preproj_attn_2023-07-04_15-31-29
{'eval_loss': 2.293712854385376, 'eval_accuracy': 0.5102272727272728, 'eval_runtime': 500.3652, 'eval_samples_per_second': 3.997, 'eval_steps_per_second': 0.126, 'eval_multicode_k': 1, 'eval_dead_code_fraction/layer0': 1.0, 'eval_MSE/layer0': 0.0, 'eval_input_norm/layer0': 0.0, 'eval_output_norm/layer0': 0.0, 'eval_dead_code_fraction/layer1': 1.0, 'eval_MSE/layer1': 0.0, 'eval_input_norm/layer1': 0.0, 'eval_output_norm/layer1': 0.0, 'eval_dead_code_fraction/layer2': 1.0, 'eval_MSE/layer2': 0.0, 'eval_input_norm/layer2': 0.0, 'eval_output_norm/layer2': 0.0, 'eval_dead_code_fraction/layer3': 1.0, 'eval_MSE/layer3': 0.0, 'eval_input_norm/layer3': 0.0, 'eval_output_norm/layer3': 0.0, 'eval_dead_code_fraction/layer4': 1.0, 'eval_MSE/layer4': 0.0, 'eval_input_norm/layer4': 0.0, 'eval_output_norm/layer4': 0.0, 'eval_dead_code_fraction/layer5': 1.0, 'eval_MSE/layer5': 0.0, 'eval_input_norm/layer5': 0.0, 'e

In [ ]:
cb_at = "attn_preproj"
ccb = "_ccb0"
# ft_tkns = features_to_tokens(f"layer0_{cb_at}{ccb}", cb_acts, num_codes=num_codes)


In [ ]:
def tokens_common_across_codes(ft_tkns, codes):
    all_ft_tkns = [ft_tkns[c] for c in codes]
    return set.intersection(*[set(ft_tkns[c]) for c in codes])


In [ ]:
ccb = "_ccb13"
ft_tkns17_13 = features_to_tokens(f"layer17_{cb_at}{ccb}", cb_acts, num_codes=num_codes)


In [ ]:
def print_dataset_example(dataset, example_idx=0):
    print(tokenizer.decode(dataset[example_idx]["input_ids"]))

In [ ]:
# print_ft_tkns(ft_tkns17_13,n=5,indices=cache_a[get_cb_layer_name("attn", 17, 13)][0, -1],tokenizer=tokenizer,tokens=tokens,max_examples=100)
print_ft_tkns(ft_tkns17_13,n=5,start=0,stop=100,tokenizer=tokenizer,tokens=tokens,max_examples=20)

In [71]:
inp = "The movie"
gen = cb_model.generate(inp, max_new_tokens=50, do_sample=True, temperature=0.7)
print(gen)
_, cache = cb_model.run_with_cache(gen)

  0%|          | 0/50 [00:00<?, ?it/s]

The movie, was originally submitted to the Berlinerff office on April 19, 2012 before director Alexander Wittgenstein. On April 26, 2012, the film was screened at the Natural History Museum in Philadelphia, which was the first time an American science fiction film has


In [ ]:
skips = 0
for layer in range(cb_model.cfg.n_layers):
    for head in range(cb_model.cfg.n_heads):
        codes, occ_freq = common_codes_in_cache(cache[get_cb_layer_name("attn", layer, head)], threshold=0.8)
        if len(codes) == 0:
            continue
        
        if (layer, head) not in ft_tkns:
            ft_tkns[(layer, head)] = features_to_tokens(f"layer{layer}_{cb_at}_ccb{head}", cb_acts, num_codes=num_codes)
        freq, acts = print_ft_tkns(ft_tkns[(layer, head)],n=3,indices=codes,tokenizer=tokenizer,tokens=tokens,max_examples=10)
        for i in range(len(freq)):
            if freq[i] > 30:
                output = f"Skipping Code: {codes[i].item()}, Layer: {layer}, Head:, {head}, Occ Freq: {occ_freq[i].item():.1f}, Train Act Freq: {freq[i]}"
                output = colored(output, "blue")
                print(output)
                skips += 1
                continue
            output = f"Code: {codes[i].item()}, Layer: {layer}, Head:, {head}, Occ Freq: {occ_freq[i].item():.1f}, Train Act Freq: {freq[i]}"
            print(output)
            print(acts[i])

print(f"Skipped {skips} codes")

In [ ]:
Layer: 3 Head: 7 Freq: 0.95 code: 841
Layer: 7 Head: 11 Freq: 0.98 code: 5806 (sus)
Layer: 9 Head: 14 Freq: 0.84 code: 199
Layer: 18 Head: 2 Freq: 0.86 code: 3545
Layer: 18 Head: 10 Freq: 0.95 code: 1666
Layer: 20 Head: 1 Freq: 0.82 code: 5263

# The football
Layer: 3 Head: 7 Freq: tensor([0.8654, 0.8077], device='cuda:0') code: tensor([8967, 8643], device='cuda:0')
code - 8967, mapped to 1.6% tokens (sports)
code - 8643, mapped to 3.3% tokens
Layer: 16 Head: 7 Freq: tensor([0.8077], device='cuda:0') code: tensor([6275], device='cuda:0')
code - 6275, mapped to 0.7 tokens
Layer: 16 Head: 15 Freq: tensor([0.8462], device='cuda:0') code: tensor([7679], device='cuda:0')
code - 7679, mapped to 0.7 tokens
Layer: 17 Head: 13 Freq: tensor([0.8269, 0.8077], device='cuda:0') code: tensor([2323, 8800], device='cuda:0')
code - 2323, mapped to 1.3 tokens
code - 8800, mapped to 1.0 tokens
Layer: 21 Head: 5 Freq: tensor([0.8846], device='cuda:0') code: tensor([6123], device='cuda:0')
code - 6123, mapped to 0.0 tokens

Code: 1111, Layer: 12, Head:, 0, Occ Freq: 0.8703703880310059, Train Act Freq: 0.86767578125


In [19]:
def patch_in_codes(run_cb_ids, hook, pos, code, code_pos=None):
    """Patch in the `code` at `run_cb_ids`."""
    pos = slice(None) if pos is None else pos
    code_pos = slice(None) if code_pos is None else code_pos

    if code_pos == "append":
        assert pos == slice(None)
        run_cb_ids = F.pad(run_cb_ids, (0, 1), mode="constant", value=code)
    if isinstance(pos, typing.Iterable) or isinstance(pos, typing.Iterable):
        for p in pos:
            run_cb_ids[:, p, code_pos] = code
    else:
        run_cb_ids[:, pos, code_pos] = code
    return run_cb_ids


In [31]:
CodeInfoTuple = namedtuple(
    "CodeInfoTuple", ["code", "cb_at", "layer", "head", "pos", "code_pos"]
)

pos = None
# code_pos = "append"
code_pos = -1
list_of_arg_tuples = [
    CodeInfoTuple(841, "attn", 3, 7, pos, code_pos), 
    # CodeInfoTuple(5806, "attn", 7, 11, pos, code_pos),
    CodeInfoTuple(199, "attn", 9, 14, pos, code_pos),
    CodeInfoTuple(3545, "attn", 18, 2, pos, code_pos),
    CodeInfoTuple(1666, "attn", 18, 10, pos, code_pos),
    CodeInfoTuple(5263, "attn", 20, 1, pos, code_pos),
]
# list_of_arg_tuples = [
#     CodeInfoTuple(8643, "attn", 3, 7, pos, code_pos), 
#     CodeInfoTuple(6275, "attn", 16, 7, pos, code_pos),
#     CodeInfoTuple(7679, "attn", 16, 15, pos, code_pos),
#     CodeInfoTuple(2323, "attn", 17, 13, pos, code_pos),
#     CodeInfoTuple(6123, "attn", 21, 5, pos, code_pos),
# ]
import typing
text_input = "The war was fought"
# gen = generate_with_codes(text_input, cb_model, list_of_arg_tuples=[], generate_kwargs={"max_new_tokens":50,"do_sample":True,"temperature":0.7})

# gen = generate_with_codes(text_input, cb_model, list_of_arg_tuples=list_of_arg_tuples, generate_kwargs={"max_new_tokens":50,"do_sample":True,"temperature":0.7})


In [42]:
def suppress_codes_in_gen(input, code_info_tuples, cb_model, generate_kwargs=None):
    """Suppresses the codebook features in the generated text."""
    generate_kwargs = {} if generate_kwargs is None else generate_kwargs
    cb_model.reset_hook_kwargs()
    for code_info in code_info_tuples:
        attn_or_mlp = 0 if code_info.cb_at == "attn" else 1
        cb_model.all_codebooks[code_info.layer][attn_or_mlp].codebook[code_info.head].set_hook_kwargs(keep_k_codes=False)
        # cb_model.all_codebooks[code_info.layer][attn_or_mlp].codebook[code_info.head].hook_kwargs["disable_codes"].append(code_info.code)
        cb_model.all_codebooks[code_info.layer][attn_or_mlp].codebook[code_info.head].set_hook_kwargs(disable_topk=8)
        

    gen = cb_model.generate(input, **generate_kwargs)
    print(gen)
    return gen

text_input = "The game"
gen = suppress_codes_in_gen(text_input, list_of_arg_tuples, cb_model, generate_kwargs={"max_new_tokens":50,"do_sample":True,"temperature":0.7})

  0%|          | 0/50 [00:00<?, ?it/s]

The game was released for the PlayStation Portable. 
 = = Reception = = 
 The game received positive reviews from critics, who praised the storyline, the graphics, and gameplay. While some called the game's graphics average, some criticized the plot,


In [43]:
logits, cache = cb_model.run_with_cache("The game")
layer, head = 9, 14
ft_tkns[(layer, head)] = features_to_tokens(f"layer{layer}_{cb_at}_ccb{head}", cb_acts, num_codes=num_codes)
indices = cache[get_cb_layer_name("attn", layer, head)][0, -1]
freq, acts = print_ft_tkns(ft_tkns[(layer, head)],n=3,indices=indices,tokenizer=tokenizer,tokens=tokens,max_examples=10)
print(acts)

100%|██████████| 2000/2000 [00:03<00:00, 514.09it/s]


code - -1, mapped to 0.1% tokens
3:  de 138 @. ... fferdam, filled with cellulose, ...  to a height of 1 @.
9: . He concluded that ... . Poniewozik described
18:  manual diving plane controls ... s alphanumeric classification system ...  at Philadelphia on 26 May 1922. ...  ship was struck from the Naval V
32:  crises in the English ...  the Black Death, there were efforts by the authorities to stem the upward pressure on wages and prices, with parliament passing the emergency Ordinance of
35:  on the Japanese television ...  = = = Context = = = 
 ...  South Africa Company were granted a Royal ... , misrepresented or simply ignored. Lobeng ...  in 1893. The resulting conflict ended ...  1894, and the subsequent submission of
37:  attempt to recover their ...  or have turned themselves in. J
41: unk> in South ...  = = = Monitoring = = = 
 Established on Kīlauea in 1912, the Hawaiian Volcano Observatory ( HVO ), presently a branch of the United States Geological Survey, is the primary or

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│   2 layer, head = 9, 14                                                                          │
│   3 ft_tkns[(layer, head)] = features_to_tokens(f"layer{layer}_{cb_at}_ccb{head}", cb_acts,      │
│   4 indices = cache[get_cb_layer_name("attn", layer, head)][0, -1]                               │
│ ❱ 5 freq, acts = print_ft_tkns(ft_tkns[(layer, head)],n=3,indices=indices,tokenizer=tokenize     │
│   6 print(acts)                                                                                  │
│   7                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: cannot unpack non-iterable NoneType object

In [ ]:
total_tokens = len(tokens) * len(tokens[0])
print(total_tokens)

In [ ]:
torch.unique(torch.randint(0, 10, (10, 10)), return_counts=True)

In [ ]:
print_ft_tkns(ft_tkns,n=5,start=0,stop=10,tokenizer=tokenizer,tokens=tokens,max_examples=100)

In [ ]:
seq = orig_cb_model.generate(do_sample=True, max_length=100, temperature=0.7)
tokenizer.decode(seq[0].tolist(), skip_special_tokens=True)

In [ ]:
cb_model.generate("The football club Manchester", max_new_tokens=5, do_sample=False)

In [ ]:
cb_model.model.cfg.device = device

In [ ]:
example_prompt = "Washington is the capital of The United"
example_answer = " States"

utils.test_prompt(
    example_prompt,
    example_answer,
    cb_model,
    prepend_bos=False,
    prepend_space_to_answer=False,
    top_k=10,
)

In [ ]:
example_prompt = "club Manchester"
example_answer = " United"

utils.test_prompt(
    example_prompt,
    example_answer,
    cb_model,
    prepend_bos=False,
    prepend_space_to_answer=False,
    top_k=10,
)

In [ ]:
input_a = "The football club Real"
input_b = "The football club Manchester"

input_a = cb_model.to_tokens(input_a, prepend_bos=False).to("cuda")
input_b = cb_model.to_tokens(input_b, prepend_bos=False).to("cuda")

logits_a, cache_a = cb_model.run_with_cache(input_a)
logits_b, cache_b = cb_model.run_with_cache(input_b)

print(f"JSD(a, b) = {JSD(logits_a, logits_b)}")

In [ ]:
input_a = "Washington is the capital of The United"
input_b = "London is the capital of The United"

input_a = cb_model.to_tokens(input_a, prepend_bos=False).to("cuda")
input_b = cb_model.to_tokens(input_b, prepend_bos=False).to("cuda")

logits_a, cache_a = cb_model.run_with_cache(input_a)
logits_b, cache_b = cb_model.run_with_cache(input_b)

print(f"JSD(a, b) = {JSD(logits_a, logits_b)}")

In [ ]:
cache_a[get_cb_layer_name("attn", 17, 13)][0, -1]

In [ ]:
[0,17,19,20]

In [ ]:
# for layer_idx in range(cb_model.cfg.n_layers):
#     n_layers = 1
for layer_idx in range(1):
    # n_layers = cb_model.cfg.n_layers
    n_layers = 4
    cb_at = ( ["attn"] * cb_model.cfg.n_heads ) * n_layers
    head = (list(range(cb_model.cfg.n_heads))) * n_layers
    # layer = [layer_idx] * len(cb_at)
    # layer = range_over_repeat(n_layers, repeat=cb_model.cfg.n_heads)
    layer = range_over_repeat([0,17,19,20], repeat=cb_model.cfg.n_heads)
    code = [cache_b[get_cb_layer_name(cb_at[i], layer[i], head[i])][0, -1, :] for i in range(len(cb_at))]
    # ind = [0,1,2,3,4,5]
    ind = range(len(cb_at))
    mod_logits, mod_cache = run_with_codes(
        input_a,
        cb_model,
        [code[i] for i in ind],
        [cb_at[i] for i in ind],
        [layer[i] for i in ind],
        [head[i] for i in ind],
        pos=[-1],
    )
    print("Layer:", layer_idx)
    print(logits_to_pred(mod_logits, tokenizer, k=5))
    print(f"JSD(a <- b, b) = {JSD(mod_logits, logits_b, pos=-1)}")


In [ ]:
js_divs = torch.zeros((cb_model.cfg.n_layers, cb_model.cfg.n_heads))
for layer_idx in tqdm(range(cb_model.cfg.n_layers)):
    for head_idx in range(cb_model.cfg.n_heads):
        # n_layers = cb_model.cfg.n_layers
        cb_at = ["attn"]
        head = [head_idx]
        layer = [layer_idx] * len(cb_at)
        # layer = range_over_repeat(n_layers, repeat=cb_model.cfg.n_heads)
        # layer = range_over_repeat([0,17,19,20], repeat=cb_model.cfg.n_heads)
        code = [cache_b[get_cb_layer_name(cb_at[i], layer[i], head[i])][0, -1, :] for i in range(len(cb_at))]
        # ind = [0,1,2,3,4,5]
        ind = range(len(cb_at))
        mod_logits, mod_cache = run_with_codes(
            input_a,
            cb_model,
            [code[i] for i in ind],
            [cb_at[i] for i in ind],
            [layer[i] for i in ind],
            [head[i] for i in ind],
            pos=[-1],
        )
        # print("Layer, Head:", layer_idx, head_idx)
        # print(logits_to_pred(mod_logits, tokenizer, k=5))
        # print(f"JSD(a <- b, b) = {JSD(mod_logits, logits_b, pos=-1)}")
        js_divs[layer_idx, head_idx] = JSD(mod_logits, logits_b, pos=-1)

# make the color map range start from 0
imshow(js_divs, xaxis="head", yaxis="layer")



In [ ]:
imshow(js_divs, xaxis="head", yaxis="layer")

In [ ]:
js_divs.sort(axis=1)

In [ ]:
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    """Show an image."""
    px.imshow(
        tl_utils.to_numpy(tensor),
        color_continuous_scale="RdBu",
        labels={"x": xaxis, "y": yaxis},
        **kwargs,
    ).show(renderer)



In [ ]:
[(17, 13)]

In [ ]:
def range_over_repeat(end_or_list, repeat=1):
    
    if isinstance(end_or_list, int):
        end_or_list = range(end_or_list)
    l = []
    for i in end_or_list:
        l += [i] * repeat
    return l